In [3]:
import pickle
import statistics 

function_to_genes_dict = pickle.load(open("../data/function_to_genes_all_in_simplified.p","rb"))

def get_function_to_age_dict(gene_to_age_dict,function_to_genes_dict,filtered_by = None):
    if filtered_by != None:
        gene_to_consensus_inaccuracy_score_dict = pickle.load(open('../data/gene_to_%s_dict.p'%filtered_by,'rb'))
        median_inaccuracy = statistics.median(list(gene_to_consensus_inaccuracy_score_dict.values()))
    
    function_to_age_dict = {}
    for func in function_to_genes_dict:
        if func == "Term": #TODO: understand why some bad data sometimes
            continue 
        # TODO: make robust against functions with genes not given in dataset
        if filtered_by == None:
            genes = function_to_genes_dict[func.strip()]
        else:
            genes = [gene for gene in function_to_genes_dict[func.strip()] if gene in gene_to_consensus_inaccuracy_score_dict and gene_to_consensus_inaccuracy_score_dict[gene] < median_inaccuracy]
        ages = list(map(lambda x: gene_to_age_dict[x.strip()] , genes)) 
        function_to_age_dict[func] = ages
    #print function_to_age_dict
    return function_to_age_dict


for filtered_by in [None,"entropy","node_error","bimodality"]:
    add_string_to_fname = ""
    if filtered_by != None:
        add_string_to_fname = "_filtered_by_%s"%filtered_by
    # for data from multiple datasets
    dataset_to_gene_to_age_dict = pickle.load(open("../data/dataset_to_gene_to_age_dict.p","rb"))  
    for dataset in dataset_to_gene_to_age_dict:
        gene_to_age_dict = dataset_to_gene_to_age_dict[dataset]
        function_to_age_dict = get_function_to_age_dict(gene_to_age_dict, function_to_genes_dict,filtered_by)
        pickle.dump(function_to_age_dict,open("../data/function_to_age_%s_dict%s.p"%(dataset,add_string_to_fname),"wb"))


    # for simplified data
    gene_to_age_simplified_dict = pickle.load(open("../data/gene_to_age_simplified_dict.p","rb"))
    function_to_age_dict = get_function_to_age_dict(gene_to_age_simplified_dict, function_to_genes_dict,filtered_by)
    pickle.dump(function_to_age_dict,open("../data/function_to_age_simplified_dict%s.p"%add_string_to_fname,"wb"))